In [7]:
import numpy as np
import altair as alt
import pandas as pd


In [8]:
!pip install vega-datasets

In [9]:
from vega_datasets import data

In [28]:
world_data = pd.read_csv('V-Dem-CPD-Party-V2.csv')

world_data.head()

,v2paenname,v2paorname,v2pashname,v2paid,pf_party_id,party_gaps,pf_url,country_name,histname,country_id,...,ep_people_vs_elite,ep_v7_lib_cons_saliency,ep_type_populism,ep_type_populist_values,ep_v8_popul_rhetoric,ep_v9_popul_saliency,e_regiongeo,e_regionpol,e_regionpol_6C,GPS_ID
0,Party of the Democratic Revolution,NaN,PRD,216,216.0,NaN,https://partyfacts.herokuapp.com/data/partycod...,Mexico,United Mexican States,3,...,NaN,NaN,NaN,NaN,NaN,NaN,17,2,2,NaN
1,Party of the Democratic Revolution,NaN,PRD,216,216.0,NaN,https://partyfacts.herokuapp.com/data/partycod...,Mexico,United Mexican States,3,...,NaN,NaN,NaN,NaN,NaN,NaN,17,2,2,NaN
2,Party of the Democratic Revolution,NaN,PRD,216,216.0,NaN,https://partyfacts.herokuapp.com/data/partycod...,Mexico,United Mexican States,3,...,NaN,NaN,NaN,NaN,NaN,NaN,17,2,2,NaN
3,Party of the Democratic Revolution,NaN,PRD,216,216.0,NaN,https://partyfacts.herokuapp.com/data/partycod...,Mexico,United Mexican States,3,...,NaN,NaN,NaN,NaN,NaN,NaN,17,2,2,NaN
4,Party of the Democratic Revolution,NaN,PRD,216,216.0,NaN,https://partyfacts.herokuapp.com/data/partycod...,Mexico,United Mexican States,3,...,NaN,NaN,NaN,NaN,NaN,NaN,17,2,2,NaN


In [29]:
print(world_data.shape)

(11898, 384)


In [30]:
world_data = world_data[world_data['year'] > 1970]

In [31]:
print(world_data.shape)

(7429, 384)


In [32]:
eu_countries = ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czechia', 'Denmark',
  'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Ireland',
  'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands',
  'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden']

In [33]:
eu_data = world_data.loc[world_data['country_name'].isin(eu_countries)]

eu_data.head()

,v2paenname,v2paorname,v2pashname,v2paid,pf_party_id,party_gaps,pf_url,country_name,histname,country_id,...,ep_people_vs_elite,ep_v7_lib_cons_saliency,ep_type_populism,ep_type_populist_values,ep_v8_popul_rhetoric,ep_v9_popul_saliency,e_regiongeo,e_regionpol,e_regionpol_6C,GPS_ID
232,Farmers' League / Centre Party,NaN,B/C,199,199.0,NaN,https://partyfacts.herokuapp.com/data/partycod...,Sweden,Kingdom of Sweden,5,...,NaN,NaN,NaN,NaN,NaN,NaN,2,5,5,NaN
233,Farmers' League / Centre Party,NaN,B/C,199,199.0,NaN,https://partyfacts.herokuapp.com/data/partycod...,Sweden,Kingdom of Sweden,5,...,NaN,NaN,NaN,NaN,NaN,NaN,2,5,5,NaN
234,Farmers' League / Centre Party,NaN,B/C,199,199.0,NaN,https://partyfacts.herokuapp.com/data/partycod...,Sweden,Kingdom of Sweden,5,...,NaN,NaN,NaN,NaN,NaN,NaN,2,5,5,NaN
235,Farmers' League / Centre Party,NaN,B/C,199,199.0,NaN,https://partyfacts.herokuapp.com/data/partycod...,Sweden,Kingdom of Sweden,5,...,NaN,NaN,NaN,NaN,NaN,NaN,2,5,5,NaN
236,Farmers' League / Centre Party,NaN,B/C,199,199.0,NaN,https://partyfacts.herokuapp.com/data/partycod...,Sweden,Kingdom of Sweden,5,...,NaN,NaN,NaN,NaN,NaN,NaN,2,5,5,NaN


In [34]:
print(eu_data.shape)

(1769, 384)


In [35]:
print(eu_data.shape)

(1769, 384)


In [39]:
world_data['v2paseatpercentage'] = world_data['v2panumbseat'] / world_data['v2patotalseat']

print(world_data['v2paseatpercentage'])

0        0.082
1        0.142
2        0.250
3        0.130
4        0.194
         ...  
11892    0.560
11893    0.480
11894    0.320
11895    0.380
11896    0.440
Name: v2paseatpercentage, Length: 7429, dtype: float64


In [40]:
print(world_data.shape)
print(eu_data.shape)

(7429, 385)
(1769, 384)


In [43]:
world = data.world_110m.url
world

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/world-110m.json'

In [44]:
missing_values_count = world_data.isnull().sum()

print(missing_values_count['year'])

# print(missing_values_count['v2paminor'])

0


In [45]:
# Columns with no missing data
world_data.columns[world_data.isnull().any() == False]

Index(['v2paenname', 'v2paid', 'country_name', 'histname', 'country_id',
       'country_text_id', 'year', 'historical_date', 'codingstart',
       'codingend', 'project', 'gap_index', 'e_regiongeo', 'e_regionpol',
       'e_regionpol_6C'],
      dtype='object')

In [46]:
test = eu_data.dropna(subset=['v2paimmig', 'v2paminor', 'year', 'country_name'])
print(test.shape)

(1606, 384)


In [47]:
alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
    fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
).project(
    type= 'mercator',
    scale= 350,                          # Magnify
    center= [20,50],                     # [lon, lat]
    clipExtent= [[0, 0], [400, 300]],
).properties(
    width=400, 
    height=300,
)

alt.Chart(...)

In [48]:
import altair as alt
from vega_datasets import data

sphere = alt.sphere()
graticule = alt.graticule(step=[10, 10])
lats = alt.sequence(start=-30, stop=71, step=10, as_='lats')
lons = alt.sequence(start=-90, stop=91, step=10, as_='lons')

width = 600
height = 450

# Source of land data
source = alt.topo_feature(data.world_110m.url, 'countries')

# Layering and configuring the components 
base = alt.layer(
    alt.Chart(sphere).mark_geoshape(fill='none'), 
    alt.Chart(graticule).mark_geoshape(stroke='gray', strokeWidth=0.5), 
    alt.Chart(source).mark_geoshape(fill='lightgray', stroke='gray')    
).properties(width=width, height=height)

projections = {
#     "Bonne": {
#         "type": "bonne",
#         "center": [0, 52],
#         "rotate": [-20, 0],
#         "parallel": 52,
#         "translate": [width/2, height/2],
#         "scale": 700,
#         "precision": 0.1
#     },
    "Equidistanc conic": {
        "type": "conicEquidistant",
        "center": [0, 52],
        "rotate": [-20, 0],        
#         "parallels": [35, 65],
        "translate": [width/2, height/2],
        "scale": 700,
        "precision": 0.1        
    },
    "Lambert conformal conic": {
        "type": "conicConformal",
        "center": [0, 52],
        "rotate": [-20, 0],
#         "parallels": [35, 65],
        "translate": [width/2, height/2],
        "scale": 700,
        "precision": 0.1        
    },
    "Albers": {
        "type": "albers",
        "center": [0, 52],
        "rotate": [-20, 0],
#         "parallels": [35, 65],
        "translate": [width/2, height/2],
        "scale": 700,
        "precision": 0.1        
    },
    "Lambert azimuthal equal-area projection": {
        "type": "azimuthalEqualArea",        
        "rotate": [-20, -52],
        "translate": [width/2, height/2],
        "scale": 700,
        "precision": 0.1        
    }    
}

charts = [base.properties(projection=projections[key], title=key) for key in projections.keys()]
alt.concat(*charts, columns=2)

alt.ConcatChart(...)

In [49]:
alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
    fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
).project(
    type='albers',
    center=[0,52],
    rotate=[-20,0],
    translate=[300, 225],
    scale=700,
    precision=0.1
).properties(
    width=600, 
    height=450,
)

alt.Chart(...)

In [311]:
eu_count = eu_data.groupby(['country_name', 'year']).size().reset_index(name='count')

eu_count.head(20)

,country_name,year,count
0,Austria,1971,3
1,Austria,1975,3
2,Austria,1979,3
3,Austria,1983,3
4,Austria,1986,3
5,Austria,1990,3
6,Austria,1994,5
7,Austria,1995,4
8,Austria,1999,4
9,Austria,2002,4


In [312]:
print(eu_count.shape)

(316, 3)


In [371]:
debug_count = eu_data.groupby(['country_name']).size().reset_index(name='count')
debug_count.rename(columns={'country_name': 'country'}, inplace=True)

debug_count.head(20)

,country,count
0,Austria,59
1,Belgium,116
2,Bulgaria,93
3,Croatia,66
4,Cyprus,40
5,Denmark,105
6,Estonia,58
7,Finland,73
8,France,58
9,Germany,84


In [358]:
print(debug_count.shape)

(26, 2)


In [375]:
# slider2 = alt.binding_range(min=eu_data['year'].min(), 
#                             max=eu_data['year'].max(), step=1, name='Current Year: ')

# yearselector = alt.selection_point(
#     name="YearSelector",
#     fields=['year'],
#     bind=slider2,
#     value=[{'year': 2000}]
# )


base = alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
    fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
).transform_lookup(
    lookup='id', from_=alt.LookupData(data=debug_count, key='country', fields=['count'])
).encode(
    alt.Color('count:Q',
              scale=alt.Scale(domain=[0, 150], clamp=True), 
              legend=alt.Legend()
              ),
    alt.Tooltip('count:Q')
).project(
    type='albers',
    center=[0,52],
    rotate=[-20,0],
    translate=[300, 225],
    scale=700,
    precision=0.1
).properties(
    width=600, 
    height=450,
).configure_view(
    stroke=None
)
# .add_params(
#     yearselector
# ).transform_filter(
#     yearselector
# )

base

/opt/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [343]:
# slider2 = alt.binding_range(min=eu_data['year'].min(), 
#                             max=eu_data['year'].max(), step=1, name='Current Year: ')

# yearselector = alt.selection_point(
#     name="YearSelector",
#     fields=['year'],
#     bind=slider2,
#     value=[{'year': 2000}]
# )


base = alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
    stroke='#706545', strokeWidth=0.5
).encode(
).project(
    type='albers',
    center=[0,52],
    rotate=[-20,0],
    translate=[300, 225],
    scale=700,
    precision=0.1
).properties(
    width=600, 
    height=450,
)

choro = alt.Chart(debug_count).mark_geoshape(
    fill='#2a1d0c', stroke='#706545'
).encode(
    alt.Color('count:Q',
            #   scale=alt.Scale(domain=[0, 2], clamp=True), 
            #   legend=alt.Legend()
              ),
    # alt.Tooltip('rate:Q', format='.0%')
).project(
    type='albers',
    center=[0,52],
    rotate=[-20,0],
    translate=[300, 225],
    scale=700,
    precision=0.1
).properties(
    width=600, 
    height=450,
)

# .configure_view(
#     stroke=None
# )
# .add_params(
#     yearselector
# ).transform_filter(
#     yearselector
# )

base + choro

# # Add Base Layer
# base = alt.Chart(geodata, title = title).mark_geoshape(
#     stroke='black',
#     strokeWidth=1
# ).encode(
# ).properties(
#     width=800,
#     height=800
# )
# # Add Choropleth Layer
# choro = alt.Chart(geodata).mark_geoshape(
#     fill='lightgray',
#     stroke='black'
# ).encode(
#     alt.Color(color_column, 
#                 type='quantitative', 
#                 scale=alt.Scale(scheme=color_scheme),
#                 title = "Building Counts"),
#     tooltip=tooltip
# )

alt.LayerChart(...)

## Notes:

maybe use all countries in europe, not just EU